In [1]:
!pip install -q marker-pdf
!pip install -q torch torchvision
!pip install -q langchain langchain-text-splitters sentence-transformers
!pip install -q pandas numpy regex chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.9/191.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.3/177.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821

In [2]:
import torch
import chromadb
import json
import re

from chromadb.utils import embedding_functions
from typing import List, Dict, Any
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import MarkdownHeaderTextSplitter
from sentence_transformers import SentenceTransformer
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.config.parser import ConfigParser
from marker.output import text_from_rendered

2025-07-09 14:59:02.294909: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752073142.494236      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752073142.552976      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# MARKDOWN FILE CONVERTER

In [3]:
# config = {
#     "output_format": "markdown",
#     "ADDITIONAL_KEY": "VALUE",
# }
# config_parser = ConfigParser(config)
# artifact_dict = create_model_dict()
# converter = PdfConverter(
#     config=config_parser.generate_config_dict(),
#     artifact_dict=create_model_dict(),
#     processor_list=config_parser.get_processors(),
#     renderer=config_parser.get_renderer(),
# )
# rendered = converter(
#     "/kaggle/input/tax-dataset-indonesia/SDSN Perpajakan 2021 2.0.pdf"
# )
# text, _, _ = text_from_rendered(rendered)

In [4]:
# output_path="/kaggle/working/sdsn_pajak_2021.md"
# with open(output_path, "w", encoding='utf-8') as f:
#     f.write(text)
# print(f"Markdown saved to: {output_path}")

# PREPROCESSING AND CHUNKING PROCESS

In [5]:
class TaxDocumentPreprocessor:
    def __init__(self):
        self.tax_terms = {
            'wajib pajak': 'wajib pajak',
            'wp': 'wajib pajak',
            'dirjen pajak': 'direktur jenderal pajak',
            'kpp': 'kantor pelayanan pajak',
            'pph': 'pajak penghasilan',
            'ppn': 'pajak pertambahan nilai',
            'pbb': 'pajak bumi dan bangunan',
            'spt': 'surat pemberitahuan',
            'pkp': 'pengusaha kena pajak',
            'bea materai': 'bea meterai'
        }
    
    def preprocess_tax_markdown(self, text: str) -> str:
        text = re.sub(r'([Pp]asal\s*)(\d+)', r'## Pasal \2', text)
        text = re.sub(r'([Aa]yat\s*\()(\d+)(\))', r'### Ayat (\2)', text)
        text = re.sub(r'([Hh]uruf\s*)([a-zA-Z])', r'#### Huruf \2', text)
        text = re.sub(r'([Aa]ngka\s*)(\d+)', r'##### Angka \2', text)
        text = re.sub(r'(BAB\s+[IVX]+)', r'# \1', text)
        for old_term, new_term in self.tax_terms.items():
            text = re.sub(r'\b' + old_term + r'\b', new_term, text, flags=re.IGNORECASE)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\n\s*\n', '\n\n', text)
        return text.strip()
    
    def extract_document_metadata(self, text: str) -> Dict[str, Any]:
        metadata = {}
        
        uu_match = re.search(r'UNDANG-UNDANG.*?NOMOR\s+(\d+)\s+TAHUN\s+(\d+)', text, re.IGNORECASE)
        if uu_match:
            metadata['nomor_uu'] = uu_match.group(1)
            metadata['tahun_uu'] = uu_match.group(2)
        
        tentang_match = re.search(r'TENTANG\s+(.*?)(?=\n|$)', text, re.IGNORECASE)
        if tentang_match:
            metadata['tentang'] = tentang_match.group(1).strip()
        
        pasal_count = len(re.findall(r'Pasal\s+\d+', text))
        metadata['jumlah_pasal'] = pasal_count
        
        bab_matches = re.findall(r'BAB\s+([IVX]+)\s+(.*?)(?=\n|BAB)', text, re.IGNORECASE)
        bab_list = [{'nomor': bab[0], 'judul': bab[1].strip()} for bab in bab_matches]
        metadata['bab_list'] = json.dumps(bab_list, ensure_ascii=False)

        return metadata

In [6]:
class TaxDocumentChunker:
    def __init__(self):
        self.preprocessor = TaxDocumentPreprocessor()
        
        self.headers_to_split_on = [
            ("#", "Judul_UU"),
            ("##", "Pasal"),
            ("###", "Ayat"),
            ("####", "Huruf"),
            ("####", "Penjelasan"),
            ("####", "Contoh"),
            ("####", "Tabel")
        ]
        
        self.tax_separators = [
            "\n\n# BAB ",         
            "\n\n## Pasal ",       
            "\n\n### Ayat ",      
            "\n\n#### Huruf ",     
            "\n\n##### Angka ",    
            "\n\n**Penjelasan",    
            "\n\n**Contoh",      
            "\n\n",                
            "\n",                  
            ". ",                 
            " "                   
        ]
    
    def create_header_based_chunks(self, text: str) -> List[Dict[str, Any]]:
        processed_text = self.preprocessor.preprocess_tax_markdown(text)
        
        markdown_splitter = MarkdownHeaderTextSplitter(
            headers_to_split_on=self.headers_to_split_on,
            strip_headers=False
        )
        
        header_chunks = markdown_splitter.split_text(processed_text)
        
        chunks = []
        for i, chunk in enumerate(header_chunks):
            chunk_data = {
                'chunk_id': f"header_{i}",
                'content': chunk.page_content,
                'metadata': chunk.metadata,
                'chunk_type': 'header_based',
                'token_count': len(chunk.page_content.split())
            }
            chunks.append(chunk_data)
        
        return chunks
    
    def create_hybrid_chunks(self, text: str, doc_metadata: Dict[str, Any]) -> List[Dict[str, Any]]:
        processed_text = self.preprocessor.preprocess_tax_markdown(text)
        
        header_chunks = self.create_header_based_chunks(processed_text)
        
        recursive_splitter = RecursiveCharacterTextSplitter(
            separators=self.tax_separators,
            chunk_size=800,
            chunk_overlap=100,
            length_function=len,
            is_separator_regex=False
        )
        
        final_chunks = []
        for chunk in header_chunks:
            if len(chunk['content']) > 800:
                sub_chunks = recursive_splitter.split_text(chunk['content'])
                for j, sub_chunk in enumerate(sub_chunks):
                    enhanced_metadata = {
                        **chunk['metadata'],
                        **doc_metadata,
                        'parent_chunk_id': chunk['chunk_id'],
                        'sub_chunk_index': j,
                        'chunk_type': 'sub_chunk',
                        'token_count': len(sub_chunk.split())
                    }
                    final_chunks.append({
                        'chunk_id': f"{chunk['chunk_id']}_sub_{j}",
                        'content': sub_chunk,
                        'metadata': enhanced_metadata
                    })
            else:
                enhanced_metadata = {
                    **chunk['metadata'],
                    **doc_metadata,
                    'chunk_type': 'header_chunk',
                    'token_count': len(chunk['content'].split())
                }
                final_chunks.append({
                    'chunk_id': chunk['chunk_id'],
                    'content': chunk['content'],
                    'metadata': enhanced_metadata
                })
        
        return final_chunks
    
    def create_semantic_chunks(self, text: str) -> List[Dict[str, Any]]:
        sections = []
        current_section = ""
        current_metadata = {}
        
        lines = text.split('\n')
        for line in lines:
            line = line.strip()
            
            if re.match(r'^#\s+BAB\s+[IVX]+', line):
                if current_section:
                    sections.append({
                        'content': current_section,
                        'metadata': current_metadata
                    })
                current_section = line + '\n'
                current_metadata = {'section_type': 'bab', 'title': line}
            
            elif re.match(r'^##\s+Pasal\s+\d+', line):
                if current_section and current_metadata.get('section_type') != 'bab':
                    sections.append({
                        'content': current_section,
                        'metadata': current_metadata
                    })
                    current_section = ""
                
                pasal_match = re.search(r'Pasal\s+(\d+)', line)
                current_metadata.update({
                    'section_type': 'pasal',
                    'pasal_number': pasal_match.group(1) if pasal_match else None,
                    'title': line
                })
                current_section += line + '\n'
            
            elif re.match(r'^###\s+Ayat\s+\(\d+\)', line):
                ayat_match = re.search(r'Ayat\s+\((\d+)\)', line)
                current_metadata.update({
                    'ayat_number': ayat_match.group(1) if ayat_match else None
                })
                current_section += line + '\n'
            
            else:
                current_section += line + '\n'
        
        if current_section:
            sections.append({
                'content': current_section,
                'metadata': current_metadata
            })
        
        chunks = []
        for i, section in enumerate(sections):
            chunks.append({
                'chunk_id': f"semantic_{i}",
                'content': section['content'],
                'metadata': section['metadata'],
                'chunk_type': 'semantic',
                'token_count': len(section['content'].split())
            })
        
        return chunks


In [7]:
def validate_metadata_types(metadata: dict):
    for k, v in metadata.items():
        if not isinstance(v, (str, int, float, bool, type(None))):
            if isinstance(v, (list, dict)):
                metadata[k] = json.dumps(v, ensure_ascii=False)
            else:
                metadata[k] = str(v)
    return metadata

In [8]:
class TaxDocumentVectorDB:
    def __init__(self, collection_name: str = "tax_documents", persist_directory: str = "/kaggle/working/chroma_db"):
        self.client = chromadb.PersistentClient(path=persist_directory)
        self.collection_name = collection_name
        self.embedding_model = SentenceTransformer('intfloat/multilingual-e5-large', device='cuda')
        
        try:
            self.collection = self.client.create_collection(collection_name)
        except:
            self.collection = self.client.get_collection(collection_name)
        
    def add_chunks_to_db(self, chunks: List[Dict[str, Any]], prefix_type="passage", batch_size=32):
        documents = [chunk['content'] for chunk in chunks]
        metadatas = [validate_metadata_types(chunk['metadata']) for chunk in chunks]
        ids = [chunk['chunk_id'] for chunk in chunks]
        prefixed_chunks = [f"{prefix_type}: {doc}" for doc in documents]

        embeddings = self.embedding_model.encode(
            prefixed_chunks,
            batch_size=batch_size,
            convert_to_tensor=True,
            normalize_embeddings=True
        ).cpu().numpy().tolist()
        
        self.collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids,
            embeddings=embeddings
        )
        
        print(f"Added {len(chunks)} chunks to vector database")

In [9]:
class ChunkQualityController:
    def __init__(self):
        self.min_chunk_size = 50
        self.max_chunk_size = 2000
        self.optimal_chunk_size = 800
    
    def validate_chunks(self, chunks: List[Dict[str, Any]]) -> Dict[str, Any]:
        quality_metrics = {
            'total_chunks': len(chunks),
            'empty_chunks': 0,
            'very_short_chunks': 0,
            'very_long_chunks': 0,
            'optimal_chunks': 0,
            'avg_chunk_size': 0,
            'pasal_coverage': 0,
            'ayat_coverage': 0,
            'context_preservation': 0
        }
        
        total_size = 0
        for chunk in chunks:
            content_length = len(chunk['content'])
            total_size += content_length
            
            if not chunk['content'].strip():
                quality_metrics['empty_chunks'] += 1
            elif content_length < self.min_chunk_size:
                quality_metrics['very_short_chunks'] += 1
            elif content_length > self.max_chunk_size:
                quality_metrics['very_long_chunks'] += 1
            elif abs(content_length - self.optimal_chunk_size) <= 200:
                quality_metrics['optimal_chunks'] += 1
            
            if 'pasal_number' in chunk.get('metadata', {}):
                quality_metrics['pasal_coverage'] += 1
            if 'ayat_number' in chunk.get('metadata', {}):
                quality_metrics['ayat_coverage'] += 1
            
            if 'pajak' in chunk['content'].lower() and content_length > 100:
                quality_metrics['context_preservation'] += 1
        
        quality_metrics['avg_chunk_size'] = total_size / len(chunks) if chunks else 0
        
        return quality_metrics
    
    def evaluate_retrieval_quality(self, chunks: List[Dict[str, Any]]) -> Dict[str, Any]:
        test_queries = [
            "tarif pajak penghasilan wajib pajak badan",
            "cara menghitung PPh pasal 21",
            "pengecualian subjek pajak",
            "sanksi keterlambatan pelaporan SPT",
            "pajak pertambahan nilai",
            "penghasilan tidak kena pajak"
        ]
        
        retrieval_scores = []
        for query in test_queries:
            relevant_chunks = []
            query_words = query.lower().split()
            
            for chunk in chunks:
                content_lower = chunk['content'].lower()
                relevance_score = sum(1 for word in query_words if word in content_lower)
                
                if relevance_score > 0:
                    relevant_chunks.append({
                        'chunk_id': chunk['chunk_id'],
                        'relevance_score': relevance_score / len(query_words),
                        'content_preview': chunk['content'][:200] + "..."
                    })
            
            relevant_chunks.sort(key=lambda x: x['relevance_score'], reverse=True)
            
            retrieval_scores.append({
                'query': query,
                'relevant_chunks_count': len(relevant_chunks),
                'top_chunks': relevant_chunks[:3],
                'precision': len(relevant_chunks) / len(chunks) if chunks else 0
            })
        
        return retrieval_scores

In [10]:
class TaxDocumentProcessor:
    def __init__(self):
        self.chunker = TaxDocumentChunker()
        self.quality_controller = ChunkQualityController()
        self.vector_db = TaxDocumentVectorDB()
        self.preprocessor = TaxDocumentPreprocessor()
    
    def process_document(self, markdown_file_path: str, strategy: str = "hybrid") -> Dict[str, Any]:
        try:
            with open(markdown_file_path, 'r', encoding='utf-8') as f:
                markdown_text = f.read()
            print(f"📖 Succesfully read markdown file with ({len(markdown_text)} character)")
        except FileNotFoundError:
            print(f"❌ File not found: {markdown_file_path}")
            return None
        except Exception as e:
            print(f"❌ Error reading file: {e}")
            return None
        
        doc_metadata = self.preprocessor.extract_document_metadata(markdown_text)
        print(f"📄 Document metadata: {doc_metadata}")
        
        if strategy == "header":
            chunks = self.chunker.create_header_based_chunks(markdown_text)
        elif strategy == "semantic":
            chunks = self.chunker.create_semantic_chunks(markdown_text)
        else:
            chunks = self.chunker.create_hybrid_chunks(markdown_text, doc_metadata)
        
        print(f"✂️ Succesfully generate {len(chunks)} chunks with {strategy} strategy")
        
        quality_metrics = self.quality_controller.validate_chunks(chunks)
        print(f"📊 Metrix quality: {quality_metrics}")
        
        retrieval_evaluation = self.quality_controller.evaluate_retrieval_quality(chunks)
        
        self.vector_db.add_chunks_to_db(chunks)
        
        return {
            'chunks': chunks,
            'document_metadata': doc_metadata,
            'quality_metrics': quality_metrics,
            'retrieval_evaluation': retrieval_evaluation,
            'processing_summary': {
                'total_chunks': len(chunks),
                'strategy_used': strategy,
                'avg_chunk_size': quality_metrics['avg_chunk_size'],
                'optimal_chunks_ratio': quality_metrics['optimal_chunks'] / len(chunks)
            }
        }
    

In [11]:
processor = TaxDocumentProcessor()
result = processor.process_document("/kaggle/input/tax-dataset-indonesia/sdsn_pajak_2021.md")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

📖 Succesfully read markdown file with (710182 character)
📄 Document metadata: {'nomor_uu': '6', 'tahun_uu': '1983', 'tentang': 'Ketentuan Umum Dan Tata Cara<br>Perpajakan Sebagaimana Telah Beberapa Kali Diubah Terakhir Dengan<br>Undang-Undang Republik Indonesia Nomor 11<br>Tahun 2020 |        |', 'jumlah_pasal': 765, 'bab_list': '[{"nomor": "I", "judul": "Ketentuan Umum                                                                                                                                                                                                                              | 1<br>1 |"}, {"nomor": "III", "judul": "Penetapan Dan Ketetapan Pajak<br>                                                                                                                                                                                                             | 28     |"}, {"nomor": "IV", "judul": "Penagihan Pajak<br>                                                                    

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

Added 1120 chunks to vector database
